In [1]:
!pip install bitsandbytes

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_name = "Qwen/Qwen3-0.6B"
# model_name = "Qwen/Qwen3-4B-Instruct-2507"
# model_name = "Qwen/Qwen3-1.7B"

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,  # 使用float16节省内存
    device_map="auto",
)


`torch_dtype` is deprecated! Use `dtype` instead!


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [4]:
for name, param in model.named_parameters():
    print(f"{name}: {param.device}")

model.embed_tokens.weight: cuda:7
model.layers.0.self_attn.q_proj.weight: cuda:7
model.layers.0.self_attn.k_proj.weight: cuda:7
model.layers.0.self_attn.v_proj.weight: cuda:7
model.layers.0.self_attn.o_proj.weight: cuda:7
model.layers.0.self_attn.q_norm.weight: cuda:7
model.layers.0.self_attn.k_norm.weight: cuda:7
model.layers.0.mlp.gate_proj.weight: cuda:7
model.layers.0.mlp.up_proj.weight: cuda:7
model.layers.0.mlp.down_proj.weight: cuda:7
model.layers.0.input_layernorm.weight: cuda:7
model.layers.0.post_attention_layernorm.weight: cuda:7
model.layers.1.self_attn.q_proj.weight: cuda:7
model.layers.1.self_attn.k_proj.weight: cuda:7
model.layers.1.self_attn.v_proj.weight: cuda:7
model.layers.1.self_attn.o_proj.weight: cuda:7
model.layers.1.self_attn.q_norm.weight: cuda:7
model.layers.1.self_attn.k_norm.weight: cuda:7
model.layers.1.mlp.gate_proj.weight: cuda:7
model.layers.1.mlp.up_proj.weight: cuda:7
model.layers.1.mlp.down_proj.weight: cuda:7
model.layers.1.input_layernorm.weight: cu

In [5]:
from datasets import load_dataset, concatenate_datasets
from datasets import Dataset

mixed_data = []

dataset = load_dataset('json', data_files='/content/data/head_200.jsonl')
# dataset = load_dataset('json', data_files='/content/data/tail_9000.jsonl')

dataset = dataset.remove_columns(["system_prompt"])

# 如果数据量大，可以分割为训练集和测试集
dataset = dataset['train'].train_test_split(test_size=0.2)
train_dataset = dataset["train"]
valid_dataset = dataset["test"]
# train_gen_dataset = Dataset.from_list(mixed_data[:8000])
# valid_gen_dataset = Dataset.from_list(mixed_data[8000:])

# train_dataset = concatenate_datasets([train_dataset, train_gen_dataset])
# valid_dataset = concatenate_datasets([valid_dataset, valid_gen_dataset])

In [6]:
def preprocess_function(example):
    """处理单个样本"""

    input_text = f"指令: {example['input']}\n响应: "
    output_text = example["output"]
    full_text = input_text + output_text

    tokenized_full = tokenizer(
        full_text,
        max_length=8192,  # 减少最大长度
        truncation=False,
        padding=False,
        return_tensors=None
    )

    tokenized_input = tokenizer(
        input_text,
        max_length=4096,
        truncation=False,
        padding=False,
        return_tensors=None
    )

    input_len = len(tokenized_input["input_ids"])
    labels = [-100] * input_len + tokenized_full["input_ids"][input_len:]

    return {
        "input_ids": tokenized_full["input_ids"],
        "attention_mask": tokenized_full["attention_mask"],
        "labels": labels
    }

tokenized_train = train_dataset.map(
    preprocess_function,
    batched=False,  # 关键：使用非批处理模式
    remove_columns=train_dataset.column_names
)

tokenized_valid = valid_dataset.map(
    preprocess_function,
    batched=False,  # 关键：使用非批处理模式
    remove_columns=valid_dataset.column_names
)

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/models/sfted/qwen3-0.6B",          # 输出目录
    # evaluation_strategy="steps",     # 按步数评估
    eval_steps=200,                  # 每200步评估一次
    save_strategy="steps",           # 按步数保存
    save_steps=500,                  # 每500步保存一次
    learning_rate=1e-5,              # 学习率（全参数微调使用较低学习率）
    per_device_train_batch_size=1,   # 训练批次大小（根据GPU内存调整）
    per_device_eval_batch_size=1,    # 评估批次大小
    gradient_accumulation_steps=16,   # 梯度累积（等效批次大小=16）
    num_train_epochs=2,              # 训练轮数
    weight_decay=0.01,               # 权重衰减
    warmup_ratio=0.1,                # 学习率预热比例
    logging_steps=50,                # 每50步记录日志
    report_to="none",                # 禁用报告
    # fp16=True,                       # 混合精度训练
    fp16=False,  # 混合精度训练
    bf16=False,
    tf32=False,
    dataloader_num_workers=2,
    # 多GPU优化配置
    dataloader_pin_memory=True,
    gradient_checkpointing=True,     # 梯度检查点节省内存
    # load_best_model_at_end=True,     # 训练结束时加载最佳模型
    metric_for_best_model="eval_loss",  # 根据验证损失选择最佳模型
    greater_is_better=False,         # 损失越低越好
    optim="paged_adamw_8bit",  # 使用分页的8-bit AdamW优化器（需bitsandbytes）
    max_grad_norm=1.0                # 梯度裁剪
)

In [8]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    # data_collator=data_collator,
    tokenizer=tokenizer
)
# Trainer会自动检测GPU并启用数据并行
print(f"Trainer检测到 {trainer.args.n_gpu} 个GPU")

Trainer检测到 1 个GPU


/tmp/ipython-input-2567715863.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
import os
import torch
import gc

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()
gc.collect()

trainer.train()

save_path = "/content/models/sfted/qwen3-0.6B"
trainer.save_model(save_path)

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss


In [10]:
import torch

def check_gpu_memory_usage():
    """检查所有GPU卡的内存使用情况"""

    print("=== GPU内存使用情况 ===")
    print(f"可用GPU数量: {torch.cuda.device_count()}")

    for i in range(torch.cuda.device_count()):
        # 获取GPU属性
        props = torch.cuda.get_device_properties(i)

        # 内存使用情况
        allocated = torch.cuda.memory_allocated(i) / 1024**3  # GB
        reserved = torch.cuda.memory_reserved(i) / 1024**3    # GB
        total = props.total_memory / 1024**3                  # GB
        free = total - allocated                              # GB

        # 使用率
        usage_percent = (allocated / total) * 100

        print(f"GPU {i}: {props.name}")
        print(f"  总内存: {total:.1f} GB")
        print(f"  已分配: {allocated:.2f} GB ({usage_percent:.1f}%)")
        print(f"  已保留: {reserved:.2f} GB")
        print(f"  可用内存: {free:.2f} GB")

        # 检查是否有进程占用
        if allocated > 0:
            print(f"  ⚠️ GPU {i} 已被占用")
        else:
            print(f"  ✅ GPU {i} 空闲")

# 运行检查
check_gpu_memory_usage()

=== GPU内存使用情况 ===
可用GPU数量: 8
GPU 0: NVIDIA A100-SXM4-40GB
  总内存: 39.5 GB
  已分配: 1.14 GB (2.9%)
  已保留: 11.58 GB
  可用内存: 38.35 GB
  ⚠️ GPU 0 已被占用
GPU 1: NVIDIA A100-SXM4-40GB
  总内存: 39.5 GB
  已分配: 0.00 GB (0.0%)
  已保留: 0.00 GB
  可用内存: 39.49 GB
  ✅ GPU 1 空闲
GPU 2: NVIDIA A100-SXM4-40GB
  总内存: 39.5 GB
  已分配: 0.00 GB (0.0%)
  已保留: 0.00 GB
  可用内存: 39.49 GB
  ✅ GPU 2 空闲
GPU 3: NVIDIA A100-SXM4-40GB
  总内存: 39.5 GB
  已分配: 0.00 GB (0.0%)
  已保留: 0.00 GB
  可用内存: 39.49 GB
  ✅ GPU 3 空闲
GPU 4: NVIDIA A100-SXM4-40GB
  总内存: 39.5 GB
  已分配: 0.00 GB (0.0%)
  已保留: 0.00 GB
  可用内存: 39.49 GB
  ✅ GPU 4 空闲
GPU 5: NVIDIA A100-SXM4-40GB
  总内存: 39.5 GB
  已分配: 0.00 GB (0.0%)
  已保留: 0.00 GB
  可用内存: 39.49 GB
  ✅ GPU 5 空闲
GPU 6: NVIDIA A100-SXM4-40GB
  总内存: 39.5 GB
  已分配: 0.00 GB (0.0%)
  已保留: 0.00 GB
  可用内存: 39.49 GB
  ✅ GPU 6 空闲
GPU 7: NVIDIA A100-SXM4-40GB
  总内存: 39.5 GB
  已分配: 0.00 GB (0.0%)
  已保留: 1.42 GB
  可用内存: 39.49 GB
  ✅ GPU 7 空闲


In [11]:
import os
import torch
import gc

del trainer

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()
gc.collect()


1351